# <font color="red">Warning: I don't have time over Christmas to comment this properly, or describe what I'm doing, so the following notebook shouldn't be taken as a good way of doing things!</font>

In [1]:
import re

import collections as coll

import pyrsistent as pyr

In [2]:
testInput_str='''
Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.
'''


In [3]:
with open('data/day25.txt') as fIn:
    puzzleInput_str=fIn.read()

In [4]:
input_str=puzzleInput_str

In [5]:
# First parse the program:

parsedInput_dict={'program':re.findall('''In state ([A-Z]):
  If the current value is 0:
    - Write the value (0|1).
    - Move one slot to the (left|right).
    - Continue with state ([A-Z]).
  If the current value is 1:
    - Write the value (0|1).
    - Move one slot to the (left|right).
    - Continue with state ([A-Z]).''', input_str)}

# Next find the start state:

parsedInput_dict['start_state']=re.search('Begin in state ([A-Z])\.',
                                          input_str).group(1)

parsedInput_dict['iterations']=int(re.search('checksum after (\d+) steps',
                                             
                                             input_str).group(1))


parsedInput_dict





{'iterations': 12656374,
 'program': [('A', '1', 'right', 'B', '0', 'left', 'C'),
  ('B', '1', 'left', 'A', '1', 'left', 'D'),
  ('C', '1', 'right', 'D', '0', 'right', 'C'),
  ('D', '0', 'left', 'B', '0', 'right', 'E'),
  ('E', '1', 'right', 'C', '1', 'left', 'F'),
  ('F', '1', 'left', 'E', '1', 'right', 'A')],
 'start_state': 'A'}

In [6]:
program_dict={(stateIn_chr, 0):{'write':int(w0),
                                'move':m0,
                                'next_state':s0}
               for (stateIn_chr, w0, m0, s0, w1, m1, s1) in parsedInput_dict['program']}

program_dict.update({(stateIn_chr, 1):{'write':int(w1),
                                       'move':m1,
                                       'next_state':s1}
                     for (stateIn_chr, w0, m0, s0, w1, m1, s1) in parsedInput_dict['program']})

program_dict

{('A', 0): {'move': 'right', 'next_state': 'B', 'write': 1},
 ('A', 1): {'move': 'left', 'next_state': 'C', 'write': 0},
 ('B', 0): {'move': 'left', 'next_state': 'A', 'write': 1},
 ('B', 1): {'move': 'left', 'next_state': 'D', 'write': 1},
 ('C', 0): {'move': 'right', 'next_state': 'D', 'write': 1},
 ('C', 1): {'move': 'right', 'next_state': 'C', 'write': 0},
 ('D', 0): {'move': 'left', 'next_state': 'B', 'write': 0},
 ('D', 1): {'move': 'right', 'next_state': 'E', 'write': 0},
 ('E', 0): {'move': 'right', 'next_state': 'C', 'write': 1},
 ('E', 1): {'move': 'left', 'next_state': 'F', 'write': 1},
 ('F', 0): {'move': 'left', 'next_state': 'E', 'write': 1},
 ('F', 1): {'move': 'right', 'next_state': 'A', 'write': 1}}

In [7]:
def move(programIn_dict, stateIn_chr, posIn_i, tapeIn_pv):
    '''
    Return the state, position and tape, following the application
    of program_dict to stateIn_chr, posIn_i and tapeIn_pv.
    '''

    # Let command_dict be the commands to move into the next state
    command_dict=programIn_dict[(stateIn_chr,
                                 tapeIn_pv[posIn_i])]
    
    tapeOut_pv=tapeIn_pv.set(posIn_i, command_dict['write'])
    if command_dict['move']=='left':
        posOut_i=posIn_i-1    
    elif command_dict['move']=='right':
        posOut_i=posIn_i+1
    
    # If we've gone off the end of the tape, extend it
    if posOut_i<0:
        posOut_i=0
        tapeOut_pv=pyr.pvector([0]).extend(tapeOut_pv)

    if posOut_i >= len(tapeOut_pv):
        tapeOut_pv=tapeOut_pv.append(0)

                
    return (command_dict['next_state'], posOut_i, tapeOut_pv)
            

In [8]:
o=(parsedInput_dict['start_state'], 0, pyr.pvector([0]))

for i in range(parsedInput_dict['iterations']):
    o=move(program_dict, *o)
    
o[2].count(1)

2526

And that seems to be it. Happy Christmas!